In [1]:
import numpy as np
from scipy.special import expit
from scipy.optimize import minimize

In [2]:
def relu(x):
    return np.maximum(x, 0)

def linear(x):
    return x

def neg(x):
    return -1 * x

In [3]:
ACTIVATION_FUNCTIONS = {
    0: relu,
    1: linear,
    2: np.tanh,
    3: expit,
    4: np.abs,
    5: neg
}

In [4]:
def relu_der(x):
    returned = []
    for x_ in x:
        returned.append(1 if x > 0 else 0)
    return returned

def linear_der(x):
    returned = []
    for x_ in x:
        returned.append(1)
    return returned

def tanh_der(x):
    returned = []
    for x_ in x:
        returned.append(1 / (np.cosh(x_) ** 2))
    return returned

def expit_der(x):
    returned = []
    for x_ in x:
        returned.append(expit(x_) * (1 - expit(x_)))
    return returned

def abs_der(x):
    returned = []
    for x_ in x:
        if x_:
            returned.append(1 if x > 0 else -1)
        else:
            returned.append(0)
    return returned

def neg_der(x):
    returned = []
    for x_ in x:
        returned.append(-1)
    return returned

In [5]:
DERIVATIVES = {
    relu: relu_der,
    linear: linear_der,
    np.tanh: tanh_der,
    expit: expit_der,
    np.abs: abs_der,
    neg: neg_der
}

In [6]:
class GenNode:
    """
    GenNode stores one node of the tree
    """
    def __init__(self, _input=None, parent=None, children=None, function=linear, dom=False, bias=1, coeff=1):
        """
        Initialize:
        - node parent
        - node children
        - activation function
        - dom of coeff (if really needed)
        - default bias (1)
        """
        if _input is None:
            self._input = np.array([1])
        else:
            self._input = np.copy(_input)
        self.parent = parent
        if children is None:
            self.children = set()
        else:
            self.children = set(children)
        self.function = function
        if dom:
            self.dom = DOM_FUNCTION[function]
        self.bias = bias
        self.coeff = coeff
        self._d = None
        
    def add_child(self, child_node):
        self.children.add(child_node)
        
    def add_parent(self, parent_node):
        self.parent = parent_node
        
    def delete_child(self, child_node):
        self.children.remove(child_node)
        
    def change_function(self, function):
        self.function = function   
        
    def calculate(self, _input=None):
        if len(self.children):
            result = np.sum(np.array([child.calculate(_input) for child in self.children]), axis=0)
        else:
            if _input is not None:
                result = np.copy(_input)
            else:
                result = np.copy(self._input)
        print('calculate')
        print(self.function(self.bias + self.coeff * result), self.bias + self.coeff * result, result)
        return self.function(self.bias + self.coeff * result)
    
    def calculate_arg(self, _input=None):
        if len(self.children):
            result = np.sum(np.array([child.calculate(_input) for child in self.children]), axis=0)
        else:
            if _input is not None:
                result = np.copy(_input)
            else:
                result = np.copy(self._input)
        return self.bias + self.coeff * result
    
    def calculate_next(self, _input=None):
        if len(self.children):
            result = np.sum(np.array([child.calculate(_input) for child in self.children]), axis=0)
        else:
            if _input is not None:
                result = np.copy(_input)
            else:
                result = np.copy(self._input)
        return result
    
    def clone(self, parent):
        node = GenNode(_input=self._input, parent=parent, function=self.function, bias=self.bias, coeff=self.coeff)
        return node

### Test node:

In [7]:
node = GenNode()

In [8]:
node.__dict__

{'_input': array([1]),
 'parent': None,
 'children': set(),
 'function': <function __main__.linear(x)>,
 'bias': 1,
 'coeff': 1,
 '_d': None}

In [9]:
cloned = node.clone(None)

In [10]:
cloned.__dict__

{'_input': array([1]),
 'parent': None,
 'children': set(),
 'function': <function __main__.linear(x)>,
 'bias': 1,
 'coeff': 1,
 '_d': None}

In [11]:
node = GenNode(_input=np.array([1,2,-3]), function=ACTIVATION_FUNCTIONS[4])
node.calculate()

calculate
[2 3 2] [ 2  3 -2] [ 1  2 -3]


array([2, 3, 2])

In [12]:
node.calculate()

calculate
[2 3 2] [ 2  3 -2] [ 1  2 -3]


array([2, 3, 2])

In [13]:
node = GenNode(_input=np.array([1,2,-3]), function=ACTIVATION_FUNCTIONS[2])
node.calculate()

calculate
[ 0.96402758  0.99505475 -0.96402758] [ 2  3 -2] [ 1  2 -3]


array([ 0.96402758,  0.99505475, -0.96402758])

In [14]:
parent = GenNode(_input=np.array([1,2,-3]), function=ACTIVATION_FUNCTIONS[2])
child = GenNode(_input=np.array([1,2,-3]), function=ACTIVATION_FUNCTIONS[0])

parent.add_child(child)
child.add_parent(parent)

In [15]:
parent.__dict__

{'_input': array([ 1,  2, -3]),
 'parent': None,
 'children': {<__main__.GenNode at 0x118a33b10>},
 'function': <ufunc 'tanh'>,
 'bias': 1,
 'coeff': 1,
 '_d': None}

In [16]:
child.__dict__

{'_input': array([ 1,  2, -3]),
 'parent': <__main__.GenNode at 0x118a33b50>,
 'children': set(),
 'function': <function __main__.relu(x)>,
 'bias': 1,
 'coeff': 1,
 '_d': None}

In [17]:
child.parent.function

<ufunc 'tanh'>

$sum(tanh(1+relu([2,3,-2])))$

In [18]:
parent.calculate()

calculate
[2 3 0] [ 2  3 -2] [ 1  2 -3]
calculate
[0.99505475 0.9993293  0.76159416] [3 4 1] [2 3 0]


array([0.99505475, 0.9993293 , 0.76159416])

In [19]:
print(relu([2,3,-2]))
print(np.tanh([3,4,1]))

[2 3 0]
[0.99505475 0.9993293  0.76159416]


---

In [103]:
class GenNet:
    """
    GenNet stores and calculates full net tree
    """
    def __init__(self, start_depth, predict):
        """
        Initialize:
        - all_nodes
        - start_depth
        - root
        """
        self.all_nodes = set()
        self.start_depth = start_depth
        self.root = None
        parent = None
        for i in range(start_depth):
            j = np.random.randint(len(ACTIVATION_FUNCTIONS))
            child = GenNode(parent=parent, function=ACTIVATION_FUNCTIONS[j])
            self.all_nodes.add(child)
            if parent is not None:
                parent.add_child(child)
            else:
                self.root = child
            parent = child
        self.predict = predict
        self.loss = 0

    def all_children(self, node):
        all_children = set()
        for child_node in node.children:
            all_children.update(self.all_children(child_node))

        return all_children
        
    def add_subnet(self, parent_node, child_node, subnet):
        removed_nodes = self.all_children(child_node)
        for node in removed_nodes:
            self.all_nodes.remove(node)
        self.all_nodes.remove(child_node)

        parent_node.add_child(subnet.root)
        for node in subnet.all_nodes:
            self.all_nodes.add(node)
        
    def clone_subnet(self, node, depth=0):
        subnet = GenNet(depth, predict=self.predict)
        subnet.root = node.clone(parent=None)
        subnet.all_nodes = set([subnet.root])

        for child in node.children:
            cloned_child = child.clone(parent=subnet.root)
            cloned_child.add_parent(subnet.root)
            subnet.root.add_child(cloned_child)
            subnet.all_nodes.update(self._clone_subnet(child, cloned_child))

        return subnet
        
    def _clone_subnet(self, root, cloned_root):
        subnet = set([cloned_root])

        for child in root.children:
            cloned_child = child.clone(parent=cloned_root)
            cloned_child.add_parent(cloned_root)
            cloned_root.add_child(cloned_child)
            subnet.update(self._clone_subnet(child, cloned_child))

        return subnet
    
    def change_function(self, node, function):
        node.change_function(function)

    def calculate(self, _input=None):
        result = self.root.calculate(_input)
        return result
    
    def calculate_node(self, node, _input=None):
        result = node.calculate(_input)
        return result
    
    def clone(self):
        net = self.clone_subnet(self.root, self.start_depth)
        return net

    def jac(self, _input=None):
        biases_jac = {}
        coeffs_jac = {}

        self.root._d = np.array([1])
        biases_jac, coeffs_jac = self.calculate_d(self.root, _input)

        if self.loss < 0:
            for ind in biases_jac.keys():
                biases_jac[ind] *= -1
                coeffs_jac[ind] *= -1
                
        biases_coeffs_jac = [biases_jac[key] for key in sorted(biases_jac.keys())]
        biases_coeffs_jac += [coeffs_jac[key] for key in sorted(coeffs_jac.keys())]
        print(np.array(biases_coeffs_jac).reshape(-1), ' jacobian')
        return np.array(biases_coeffs_jac).reshape(-1)
        
    def calculate_d(self, node, _input=None):
        if node.parent:
            static_part = node.parent._d
        else:
            static_part = np.array([1])
        ind = list(self.all_nodes).index(node)
        biases_jac = {}
        coeffs_jac = {}
        print('arg ', node.calculate_arg(_input))
        calculated = static_part * DERIVATIVES[node.function](node.calculate_arg(_input))
        biases_jac[ind] = calculated
        coeffs_jac[ind] = calculated * node.calculate_next(_input)
        node._d = calculated * node.coeff
        
        for child in node.children:
            new_biases_jac, new_coeffs_jac = self.calculate_d(child, _input)
            biases_jac.update(new_biases_jac)
            coeffs_jac.update(new_coeffs_jac)

        return biases_jac, coeffs_jac

    def biases_coeffs_inference(self, biases, coeffs):
        for ind, node in enumerate(list(self.all_nodes)):
            node.bias = biases[ind]
            node.coeff = coeffs[ind]
            
    def input_inference(self, _input):
        for node in self.all_nodes:
            node._input = _input

    def calculate_biases_coeffs_loss(self, biases_coeffs):
        self.biases_coeffs_inference(biases_coeffs[:len(biases_coeffs)//2], biases_coeffs[len(biases_coeffs)//2:])
        self.loss = np.abs(self.calculate() - self.predict)
        print(self.loss, ' loss')
        return self.loss

### Test net

In [22]:
net = GenNet(1, np.array([1]))

In [23]:
net.__dict__

{'all_nodes': {<__main__.GenNode at 0x118a94690>},
 'start_depth': 1,
 'root': <__main__.GenNode at 0x118a94690>,
 'predict': array([1]),
 'loss': 0}

In [24]:
net.root.__dict__

{'_input': array([1]),
 'parent': None,
 'children': set(),
 'function': <ufunc 'expit'>,
 'bias': 1,
 'coeff': 1,
 '_d': None}

In [25]:
net.calculate(_input=np.array([2,3,0]))

calculate
[0.95257413 0.98201379 0.73105858] [3 4 1] [2 3 0]


array([0.95257413, 0.98201379, 0.73105858])

In [26]:
net.change_function(net.root, relu)

In [27]:
net.calculate(_input=np.array([1,-2,0]))

calculate
[2 0 1] [ 2 -1  1] [ 1 -2  0]


array([2, 0, 1])

In [28]:
net.calculate_biases_coeffs_loss(np.array([0,1]))

calculate
[1] [1] [1]
[0]  loss


array([0])

In [29]:
net.calculate()

calculate
[1] [1] [1]


array([1])

In [30]:
net.root.__dict__

{'_input': array([1]),
 'parent': None,
 'children': set(),
 'function': <function __main__.relu(x)>,
 'bias': 0,
 'coeff': 1,
 '_d': None}

In [31]:
net = GenNet(2, [1])

In [32]:
for node in net.all_nodes:
    print(node.__dict__)

{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118a94650>, 'children': set(), 'function': <ufunc 'absolute'>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': None, 'children': {<__main__.GenNode object at 0x118ac32d0>}, 'function': <ufunc 'absolute'>, 'bias': 1, 'coeff': 1, '_d': None}


In [33]:
net.calculate(np.array([-2]))

calculate
[1] [-1] [-2]
calculate
[2] [2] [1]


array([2])

In [34]:
net = GenNet(3, [1])

In [35]:
for node in net.all_nodes:
    print(node.__dict__)

{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ac33d0>, 'children': {<__main__.GenNode object at 0x118ac37d0>}, 'function': <function linear at 0x1189d6b00>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ac3c50>, 'children': set(), 'function': <function relu at 0x1113f47a0>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': None, 'children': {<__main__.GenNode object at 0x118ac3c50>}, 'function': <ufunc 'tanh'>, 'bias': 1, 'coeff': 1, '_d': None}


In [36]:
subnet = net.clone_subnet(list(net.all_nodes)[1])

In [37]:
subnet.__dict__

{'all_nodes': {<__main__.GenNode at 0x118ac91d0>},
 'start_depth': 0,
 'root': <__main__.GenNode at 0x118ac91d0>,
 'predict': [1],
 'loss': 0}

In [38]:
subnet.root.children

set()

In [39]:
net.calculate(_input=np.array([1]))

calculate
[2] [2] [1]
calculate
[3] [3] [2]
calculate
[0.9993293] [4] [3]


array([0.9993293])

In [40]:
expit(np.tanh(2)+1) * (1 - expit(np.tanh(2)+1))

0.10789500003278905

In [41]:
jac = net.jac()

calculate
[2] [2] [1]
calculate
[3] [3] [2]
arg  [4]
calculate
[2] [2] [1]
calculate
[3] [3] [2]
calculate
[2] [2] [1]
calculate
[3] [3] [2]
calculate
[2] [2] [1]
arg  [3]
calculate
[2] [2] [1]
calculate
[2] [2] [1]
arg  [2]
[0.00134095 0.00134095 0.00134095 0.0026819  0.00134095 0.00402285]  jacobian


In [42]:
jac

array([0.00134095, 0.00134095, 0.00134095, 0.0026819 , 0.00134095,
       0.00402285])

In [45]:
net = GenNet(1, np.array([1]))
net_2 = GenNet(2, np.array([2]))
net_3 = GenNet(3, np.array([3]))
net_4 = GenNet(4, np.array([4]))

In [46]:
nets = [
    net,
    net_2,
    net_3,
    net_4
]

In [49]:
for net_ in nets:
    for node_ in net_.all_nodes:
        print(node_.__dict__)
    print()
    print(net_.calculate())
    print()

{'_input': array([1]), 'parent': None, 'children': set(), 'function': <ufunc 'absolute'>, 'bias': 1, 'coeff': 1, '_d': None}

calculate
[2] [2] [1]
[2]

{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ad18d0>, 'children': set(), 'function': <ufunc 'absolute'>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': None, 'children': {<__main__.GenNode object at 0x118ad1490>}, 'function': <function relu at 0x1113f47a0>, 'bias': 1, 'coeff': 1, '_d': None}

calculate
[2] [2] [1]
calculate
[3] [3] [2]
[3]

{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ad1350>, 'children': set(), 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ad1650>, 'children': {<__main__.GenNode object at 0x118ad1150>}, 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': None, 'children': {<__main__.GenNode object at 0

---

In [63]:
population_size = 3
depth = 10
predict = np.array([1])

In [64]:
nets = [
    GenNet(depth, predict) for _ in range(population_size)
]

In [65]:
nets

In [66]:
prob = [0.2, 0.4, 0.4]
        
action = np.random.choice(3, p=prob)

In [87]:
def mutate_net(nets, net_ind):
    net = list(nets)[net_ind].clone()
    for node_ in net.all_nodes:
        print(node_.__dict__)
    i = np.random.randint(len(net.all_nodes))
    node = list(net.all_nodes)[i]
    function_ind = np.random.randint(len(ACTIVATION_FUNCTIONS))
    new_node = GenNode(_input=node._input, parent=node, 
                       function=ACTIVATION_FUNCTIONS[function_ind], bias=node.bias, coeff=node.coeff)
    node.add_child(new_node)
    print()
    print(node.__dict__, node)
    print()
    net.all_nodes.add(new_node)
    for node_ in net.all_nodes:
        print(node_.__dict__)

    return net

In [100]:
net__1 = GenNet(3, [1])
net__2 = GenNet(3, [1])

In [101]:
mutated_net = mutate_net([net__1], 0)


{<__main__.GenNode object at 0x11a0ba590>}

subnet
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x11a0ba050>, 'children': set(), 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None}
subnet
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x11a0ba050>, 'children': set(), 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x11a0ba590>, 'children': {<__main__.GenNode object at 0x11a0ba410>}, 'function': <ufunc 'tanh'>, 'bias': 1, 'coeff': 1, '_d': None}
full subnet
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x11a0ba050>, 'children': set(), 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': None, 'children': {<__main__.GenNode object at 0x11a0ba050>}, 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': <__main__.Gen

In [78]:
for node_ in net__1.all_nodes:
    print(node_.__dict__)

{'_input': array([1]), 'parent': None, 'children': {<__main__.GenNode object at 0x11a09fa10>}, 'function': <ufunc 'expit'>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ac5610>, 'children': {<__main__.GenNode object at 0x11a09f050>}, 'function': <ufunc 'expit'>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x11a09fa10>, 'children': set(), 'function': <function linear at 0x1189d6b00>, 'bias': 1, 'coeff': 1, '_d': None}


In [102]:
for node_ in mutated_net.all_nodes:
    print(node_.__dict__, node_)

{'_input': array([1]), 'parent': <__main__.GenNode object at 0x11a0ba050>, 'children': set(), 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None} <__main__.GenNode object at 0x11a0ba410>
{'_input': array([1]), 'parent': None, 'children': {<__main__.GenNode object at 0x11a0ba050>}, 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None} <__main__.GenNode object at 0x11a0ba590>
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x11a0ba590>, 'children': {<__main__.GenNode object at 0x11a0ba410>, <__main__.GenNode object at 0x119fa5a50>}, 'function': <ufunc 'tanh'>, 'bias': 1, 'coeff': 1, '_d': None} <__main__.GenNode object at 0x11a0ba050>
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x11a0ba050>, 'children': set(), 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None} <__main__.GenNode object at 0x119fa5a50>


In [68]:
if action == 1:
    i, j = np.random.randint(len(nets), size=2)

    net_1 = list(nets)[i]
    while len(net_1.all_nodes) == 1:
        i = np.random.randint(len(nets))
        net_1 = list(nets)[i]

    net_2 = list(self.population)[j]
    while len(net_2.all_nodes) == 1:
        j = np.random.randint(len(nets))
        net_2 = list(nets)[j]

    new_net = self.cross(net_1, net_2)
    if new_net:
        self.population = list(np.concatenate((np.array(list(nets)), np.array(new_net))))

elif action == 2:
    i = np.random.randint(len(nets))
    nets.append(mutate_net(nets, i))

In [69]:
nets

In [109]:
for node_ in nets[3].all_nodes:
    print(node_.__dict__)

{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ac53d0>, 'children': {<__main__.GenNode object at 0x118ac5450>}, 'function': <function relu at 0x1113f47a0>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ac57d0>, 'children': {<__main__.GenNode object at 0x118ac5850>, <__main__.GenNode object at 0x118ac58d0>}, 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': array([-0.4231454])}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ac5410>, 'children': {<__main__.GenNode object at 0x118ac5490>}, 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': None}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ac5810>, 'children': {<__main__.GenNode object at 0x118ac5890>}, 'function': <ufunc 'tanh'>, 'bias': 1, 'coeff': 1, '_d': array([-0.00417477])}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118ac5450>, 'children': {<__main__.GenNode

In [73]:
for net_ in nets:
    for node_ in net_.all_nodes:
        print(node_.__dict__)
    print()
    print(net_.calculate(), net_.jac(), len(net_.all_nodes))
    print()

{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118af81d0>, 'children': {<__main__.GenNode object at 0x118af8250>}, 'function': <function relu at 0x1113f47a0>, 'bias': 1, 'coeff': 1, '_d': array([-0.01829339])}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118af83d0>, 'children': {<__main__.GenNode object at 0x118af8450>}, 'function': <function relu at 0x1113f47a0>, 'bias': 1, 'coeff': 1, '_d': array([-1.63593313e-05])}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118af8210>, 'children': {<__main__.GenNode object at 0x118af8310>}, 'function': <function relu at 0x1113f47a0>, 'bias': 1, 'coeff': 1, '_d': array([-0.01829339])}
{'_input': array([1]), 'parent': None, 'children': {<__main__.GenNode object at 0x118af81d0>}, 'function': <function neg at 0x1189e7830>, 'bias': 1, 'coeff': 1, '_d': array([-1])}
{'_input': array([1]), 'parent': <__main__.GenNode object at 0x118af8410>, 'children': set(), 'function': <ufunc 'expit'>, 'bias': 1, 'coe

[3.99500572] [3.99500572] [2.99500572]
arg  [4.99500572]
calculate
[2] [2] [1]
calculate
[0.99505475] [3] [2]
calculate
[-1.99505475] [1.99505475] [0.99505475]
calculate
[0.99505475] [-0.99505475] [-1.99505475]
calculate
[1.99505475] [1.99505475] [0.99505475]
calculate
[0.99500572] [2.99505475] [1.99505475]
calculate
[1.99500572] [1.99500572] [0.99500572]
calculate
[2.99500572] [2.99500572] [1.99500572]
calculate
[3.99500572] [3.99500572] [2.99500572]
calculate
[2] [2] [1]
calculate
[0.99505475] [3] [2]
calculate
[-1.99505475] [1.99505475] [0.99505475]
calculate
[0.99505475] [-0.99505475] [-1.99505475]
calculate
[1.99505475] [1.99505475] [0.99505475]
calculate
[0.99500572] [2.99505475] [1.99505475]
calculate
[1.99500572] [1.99500572] [0.99500572]
calculate
[2.99500572] [2.99500572] [1.99500572]
calculate
[3.99500572] [3.99500572] [2.99500572]
calculate
[2] [2] [1]
calculate
[0.99505475] [3] [2]
calculate
[-1.99505475] [1.99505475] [0.99505475]
calculate
[0.99505475] [-0.99505475] [-1.9

[-1.94244819] [1.94244819] [0.94244819]
calculate
[-0.73634505] [-0.94244819] [-1.94244819]
calculate
[2] [2] [1]
calculate
[0.99505475] [3] [2]
calculate
[-1.99505475] [1.99505475] [0.99505475]
calculate
[-0.75950944] [-0.99505475] [-1.99505475]
calculate
[-0.24049056] [0.24049056] [-0.75950944]
calculate
[0.75950944] [0.75950944] [-0.24049056]
calculate
[0.94244819] [1.75950944] [0.75950944]
calculate
[-1.94244819] [1.94244819] [0.94244819]
arg  [-0.94244819]
calculate
[2] [2] [1]
calculate
[0.99505475] [3] [2]
calculate
[-1.99505475] [1.99505475] [0.99505475]
calculate
[-0.75950944] [-0.99505475] [-1.99505475]
calculate
[-0.24049056] [0.24049056] [-0.75950944]
calculate
[0.75950944] [0.75950944] [-0.24049056]
calculate
[0.94244819] [1.75950944] [0.75950944]
calculate
[-1.94244819] [1.94244819] [0.94244819]
calculate
[2] [2] [1]
calculate
[0.99505475] [3] [2]
calculate
[-1.99505475] [1.99505475] [0.99505475]
calculate
[-0.75950944] [-0.99505475] [-1.99505475]
calculate
[-0.24049056] 

[3.75950944] [3.75950944] [2.75950944]
calculate
[2] [2] [1]
calculate
[0.99505475] [3] [2]
calculate
[-2] [2] [1]
calculate
[0.00494525] [-0.00494525] [-1.00494525]
calculate
[-1.00494525] [1.00494525] [0.00494525]
calculate
[-0.00494525] [-0.00494525] [-1.00494525]
calculate
[0.75950944] [0.99505475] [-0.00494525]
calculate
[1.75950944] [1.75950944] [0.75950944]
calculate
[2.75950944] [2.75950944] [1.75950944]
arg  [3.75950944]
calculate
[2] [2] [1]
calculate
[0.99505475] [3] [2]
calculate
[-2] [2] [1]
calculate
[0.00494525] [-0.00494525] [-1.00494525]
calculate
[-1.00494525] [1.00494525] [0.00494525]
calculate
[-0.00494525] [-0.00494525] [-1.00494525]
calculate
[0.75950944] [0.99505475] [-0.00494525]
calculate
[1.75950944] [1.75950944] [0.75950944]
calculate
[2.75950944] [2.75950944] [1.75950944]
calculate
[2] [2] [1]
calculate
[0.99505475] [3] [2]
calculate
[-2] [2] [1]
calculate
[0.00494525] [-0.00494525] [-1.00494525]
calculate
[-1.00494525] [1.00494525] [0.00494525]
calculate
[-

---

In [104]:
class GenGenetic:
    def __init__(self, population_size=100, max_depth=10,
                 nets_amount=10, functions=ACTIVATION_FUNCTIONS):
        self.population_size = population_size
        self.population = set()
        
        self.functions = functions
        self.max_depth = max_depth
        self.nets_amount = nets_amount
        
    def create_population(self, predict):
        for _ in range(self.population_size):
            j = np.random.randint(1, self.max_depth+1)
            self.population.add(GenNet(j, predict))
        
    def clone(self, i): ### UPDATE ENVIRONMENT
        pass
    
    def cross(self, net_1, net_2):
        net_1 = net_1.clone()
        net_2 = net_2.clone()
        
        i = np.random.randint(len(net_1.all_nodes))
        j = np.random.randint(len(net_2.all_nodes))
        
        node_1 = list(net_1.all_nodes)[i]
        node_2 = list(net_2.all_nodes)[j]
        
        while node_1.parent is None:
            i = np.random.randint(len(net_1.all_nodes))
            node_1 = list(net_1.all_nodes)[i]

        while node_2.parent is None:
            j = np.random.randint(len(net_2.all_nodes))
            node_2 = list(net_2.all_nodes)[j]
        
        subnet_1 = net_1.clone_subnet(node_1)
        subnet_2 = net_2.clone_subnet(node_2)
        
        net_1.add_subnet(node_1.parent, node_1, subnet_2)
        net_2.add_subnet(node_2.parent, node_2, subnet_1)
        
        return [net_1, net_2]
    
    def mutate(self, net_ind):
        net = list(self.population)[net_ind].clone()
        i = np.random.randint(len(net.all_nodes))
        node = list(net.all_nodes)[i]
        function_ind = np.random.randint(len(ACTIVATION_FUNCTIONS))
        new_node = GenNode(_input=node._input, parent=node, 
                           function=ACTIVATION_FUNCTIONS[function_ind], bias=node.bias, coeff=node.coeff)
        node.add_child(new_node)
        net.all_nodes.add(new_node)
        
        return net
        
    def step(self, prob=0.25):
        if not isinstance(prob, list):
            prob = [1 - prob * 2] + [prob] * 2
        
        action = np.random.choice(3, p=prob)
        print(action)
        if action == 1:
            i, j = np.random.randint(len(self.population), size=2)

            net_1 = list(self.population)[i]
            while len(net_1.all_nodes) == 1:
                i = np.random.randint(len(self.population))
                net_1 = list(self.population)[i]

            net_2 = list(self.population)[j]
            while len(net_2.all_nodes) == 1:
                j = np.random.randint(len(self.population))
                net_2 = list(self.population)[j]
                
            new_net = self.cross(net_1, net_2)
            if new_net:
                self.population = list(np.concatenate((np.array(list(self.population)), np.array(new_net))))

        elif action == 2:
            i = np.random.randint(len(self.population))
            self.population.add(self.mutate(i))
            
    def calculate_result(self, _input):
        results = []
        for net in self.population:
            results.append(net.calculate(_input))
        return results
    
    def run(self, _input, predict, num_steps=50, prob=0.25, i=1):
        self.create_population(predict)
        for step in range(num_steps):
            self.step(prob=prob)

        for net in self.population:
            x0 = [1 for _ in range(len(net.all_nodes) * 2)]
            opt_biases_coeff = minimize(net.calculate_biases_coeffs_loss, x0, method='BFGS', jac=net.jac)
            opt_biases = opt_biases_coeff[:len(opt_biases_coeff)//2]
            opt_coeffs = opt_biases_coeff[len(opt_biases_coeff)//2:]
            net.biases_coeffs_inference(opt_biases, opt_coeffs)
        
        results = self.calculate_result(_input)
        losses = np.array([loss(result) for result in results])
        best_ind = losses.argsort()[-1 * self.nets_amount:][::-1]
        self.population = np.array(self.population)[best_ind].tolist()
            
        return self.population, best_ind

In [105]:
genetic_3 = GenGenetic(population_size=1, max_depth=5)

In [107]:
_input = np.array([1])

In [108]:
genetic_3.run(_input, predict=np.array([1]), num_steps=2, prob=0)

0
0
calculate
[2. 2. 2. 2. 2. 2. 2. 2.] [2. 2. 2. 2. 2. 2. 2. 2.] [1. 1. 1. 1. 1. 1. 1. 1.]
calculate
[3. 3. 3. 3. 3. 3. 3. 3.] [3. 3. 3. 3. 3. 3. 3. 3.] [2. 2. 2. 2. 2. 2. 2. 2.]
calculate
[0.98201379 0.98201379 0.98201379 0.98201379 0.98201379 0.98201379
 0.98201379 0.98201379] [4. 4. 4. 4. 4. 4. 4. 4.] [3. 3. 3. 3. 3. 3. 3. 3.]
arg  [1.98201379 1.98201379 1.98201379 1.98201379 1.98201379 1.98201379
 1.98201379 1.98201379]
calculate
[2. 2. 2. 2. 2. 2. 2. 2.] [2. 2. 2. 2. 2. 2. 2. 2.] [1. 1. 1. 1. 1. 1. 1. 1.]
calculate
[3. 3. 3. 3. 3. 3. 3. 3.] [3. 3. 3. 3. 3. 3. 3. 3.] [2. 2. 2. 2. 2. 2. 2. 2.]
calculate
[0.98201379 0.98201379 0.98201379 0.98201379 0.98201379 0.98201379
 0.98201379 0.98201379] [4. 4. 4. 4. 4. 4. 4. 4.] [3. 3. 3. 3. 3. 3. 3. 3.]
calculate
[2. 2. 2. 2. 2. 2. 2. 2.] [2. 2. 2. 2. 2. 2. 2. 2.] [1. 1. 1. 1. 1. 1. 1. 1.]
calculate
[3. 3. 3. 3. 3. 3. 3. 3.] [3. 3. 3. 3. 3. 3. 3. 3.] [2. 2. 2. 2. 2. 2. 2. 2.]
calculate
[0.98201379 0.98201379 0.98201379 0.98201379 0.98201379 

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [229]:
genetic_2 = GenGenetic(population_size=1, max_depth=5)

In [227]:
_input

array([1])

In [230]:
genetic_2.run(_input, predict=np.array([1]), num_steps=2, prob=0)

0
0
children  {<__main__.GenNode object at 0x120206d90>}
calculate
[2. 2. 2. 2.] [2. 2. 2. 2.] [1. 1. 1. 1.]
[2. 2. 2. 2.]
calculate
[2. 2. 2. 2.] [2. 2. 2. 2.] [1. 1. 1. 1.]
children result  [[2. 2. 2. 2.]]
calculate
[2. 2. 2. 2.] [2. 2. 2. 2.] [1. 1. 1. 1.]
result  [2. 2. 2. 2.]
calculate
[2. 2. 2. 2.] [2. 2. 2. 2.] [1. 1. 1. 1.]
return  [3. 3. 3. 3.]
arg  [3. 3. 3. 3.]
children  {<__main__.GenNode object at 0x120206d90>}
calculate
[2. 2. 2. 2.] [2. 2. 2. 2.] [1. 1. 1. 1.]
[2. 2. 2. 2.]
calculate
[2. 2. 2. 2.] [2. 2. 2. 2.] [1. 1. 1. 1.]
children result  [[2. 2. 2. 2.]]
calculate
[2. 2. 2. 2.] [2. 2. 2. 2.] [1. 1. 1. 1.]
result  [2. 2. 2. 2.]
calculate
[2. 2. 2. 2.] [2. 2. 2. 2.] [1. 1. 1. 1.]
return  [3. 3. 3. 3.]
calculate
[2. 2. 2. 2.] [2. 2. 2. 2.] [1. 1. 1. 1.]
res = input [1. 1. 1. 1.]
return  [2. 2. 2. 2.]
arg  [2. 2. 2. 2.]
res = input [1. 1. 1. 1.]
return  [2. 2. 2. 2.]
[0.00986604 0.00986604 0.00986604 0.00986604 0.00986604 0.00986604
 0.00986604 0.00986604 0.01973207 0.019

ValueError: shapes (4,4) and (16,) not aligned: 4 (dim 1) != 16 (dim 0)

In [205]:
genetic = GenGenetic(population_size=1, max_depth=5)

In [201]:
new_genetic = GenGenetic(population_size=2, max_depth=3)

In [203]:
new_genetic.population

{<__main__.GenNet at 0x11f95ca50>, <__main__.GenNet at 0x12020fc10>}

In [202]:
new_genetic.run(_input, predict=np.array([1]), num_steps=2, prob=0)

0
0
arg  [1.95257413 1.95257413 1.95257413 1.95257413 1.95257413 1.95257413]
arg  [3. 3. 3. 3. 3. 3.]
arg  [2. 2. 2. 2. 2. 2.]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [206]:
genetic.run(_input, predict=np.array([1]), num_steps=2, prob=0)

0
0
children result  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
result  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
return  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
arg  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
children result  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
result  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
return  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [166]:
a = set([1,2])

In [86]:
genetic.population

{<__main__.GenNet at 0x11e48e690>, <__main__.GenNet at 0x1201d7890>}

## Test

In [58]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [59]:
ort10_coltar10 = pd.read_csv('lin_ort10_coltar10_mean.csv', header=None).values
ort10_colfeat10 = pd.read_csv('lin_ort10_colfeat10_mean.csv', header=None).values
ort20 = pd.read_csv('lin_ort20_mean.csv', header=None).values
ort5_coltar10_colfeat5 = pd.read_csv('lin_ort5_coltar10_colfeat5_mean.csv', header=None).values
ort5_coltar15 = pd.read_csv('lin_ort5_coltar15_mean.csv', header=None).values
ort10_coltar5_colfeat5 = pd.read_csv('lin_ort10_coltar5_colfeat5_mean.csv', header=None).values
ort5_colfeat15 = pd.read_csv('lin_ort5_colfeat15_mean.csv', header=None).values
ort15_colfeat5 = pd.read_csv('lin_ort15_colfeat5_mean.csv', header=None).values
ort5_coltar5_colfeat10 = pd.read_csv('lin_ort5_coltar5_colfeat10_mean.csv', header=None).values

ort10_coltar10_data = pd.read_csv('lin_ort10_coltar10_data.csv', header=None)
ort10_colfeat10_data = pd.read_csv('lin_ort10_colfeat10_data.csv', header=None)
ort20_data = pd.read_csv('lin_ort20_data.csv', header=None)
ort5_coltar10_colfeat5_data = pd.read_csv('lin_ort5_coltar10_colfeat5_data.csv', header=None)
ort5_coltar15_data = pd.read_csv('lin_ort5_coltar15_data.csv', header=None)
ort10_coltar5_colfeat5_data = pd.read_csv('lin_ort10_coltar5_colfeat5_data.csv', header=None)
ort5_colfeat15_data = pd.read_csv('lin_ort5_colfeat15_data.csv', header=None)
ort15_colfeat5_data = pd.read_csv('lin_ort15_colfeat5_data.csv', header=None)
ort5_coltar5_colfeat10_data = pd.read_csv('lin_ort5_coltar5_colfeat10_data.csv', header=None)

In [60]:
true_results = [
    ort10_coltar10.reshape(-1),
    ort10_colfeat10.reshape(-1),
    ort20.reshape(-1),
    ort5_coltar10_colfeat5.reshape(-1),
    ort5_coltar15.reshape(-1),
    ort10_coltar5_colfeat5.reshape(-1),
    ort5_colfeat15.reshape(-1),
    ort15_colfeat5.reshape(-1),
    ort5_coltar5_colfeat10.reshape(-1)
]

In [61]:
_input = np.arange(2, 2 + len(true_results[0]))

In [62]:
_input = np.array([1])

In [746]:
netty = genetic.population[1]

TypeError: 'set' object is not subscriptable

In [747]:
for node in genetic.population[1].all_nodes:
    print(node.__dict__)

TypeError: 'set' object is not subscriptable

In [748]:
np.array(netty.jac()).reshape(-1)

[array([1]), array([1])]


array([1, 1])

In [749]:
genetic = GenGenetic(population_size=1)

In [750]:
genetic.create_population([1])

In [751]:
genetic.population

{<__main__.GenNet at 0x11a9af710>}

In [752]:
genetic.run(_input, predict=np.array([1]), num_steps=2, prob=0.5)

2
2
arg  [2. 2.]
[2. 2.]
arg  [2. 2.]
next  [2. 2.]
[1]
calc  [2.]


ValueError: shapes (2,2) and (4,) not aligned: 2 (dim 1) != 4 (dim 0)

In [753]:
genetic.run(_input)

TypeError: run() missing 1 required positional argument: 'predict'